# ETL

Firstly, we have to import the right libraries

In [3]:
import pandas as pd
import numpy as np

We download the file `LA Crime_Data_from_2010_to_2019.csv` from [Kaggle](https://www.kaggle.com/jpmiller/police-violence-racial-equity) and move it to the same folder as this notebook
* Reading the csv file

In [4]:
la_crimes = pd.read_csv('Crime_Data_from_2010_to_2019.csv')
la_crimes.rename(columns={'AREA ': 'AREA'}, inplace = True)
la_crimes

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,...,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,...,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524
3,90631215,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,...,IC,Invest Cont,900.0,998.0,NaN,NaN,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3295
4,100100501,01/03/2010 12:00:00 AM,01/02/2010 12:00:00 AM,2100,1,Central,176,1,122,"RAPE, ATTEMPTED",...,IC,Invest Cont,122.0,NaN,NaN,NaN,8TH ST,SAN PEDRO ST,34.0387,-118.2488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117584,190608903,03/28/2019 12:00:00 AM,03/28/2019 12:00:00 AM,400,6,Hollywood,644,1,648,ARSON,...,IC,Invest Cont,648.0,NaN,NaN,NaN,1400 N LA BREA AV,NaN,34.0962,-118.3490
2117585,190715222,08/15/2019 12:00:00 AM,08/14/2019 12:00:00 AM,1810,7,Wilshire,701,1,331,THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER),...,IC,Invest Cont,331.0,NaN,NaN,NaN,WILLOUGHBY AV,ORLANDO AV,34.0871,-118.3732
2117586,192004409,01/06/2019 12:00:00 AM,01/06/2019 12:00:00 AM,2100,20,Olympic,2029,2,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,...,IC,Invest Cont,930.0,NaN,NaN,NaN,6TH,VIRGIL,34.0637,-118.2870
2117587,191716777,10/17/2019 12:00:00 AM,10/16/2019 12:00:00 AM,1800,17,Devonshire,1795,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,IC,Invest Cont,420.0,NaN,NaN,NaN,17200 NAPA ST,NaN,34.2266,-118.5085


## DR_NO column
* We see that the column `DR_NO` is a unique code for each row, that means each crime reported has a different code

In [5]:
la_crimes['DR_NO'].value_counts()

DR_NO
1307355      1
161912897    1
160320555    1
161117152    1
161818480    1
            ..
131617324    1
132112099    1
140204026    1
131121431    1
190805435    1
Name: count, Length: 2117589, dtype: int64

## Date Rptd and DATE OCC columns
* We see that all the dates have the time of 12:00:00 AM

In [6]:
la_crimes['Date Rptd'].str[-8:].value_counts() #selecting only the time and getting the different values of it

Date Rptd
00:00 AM    2117589
Name: count, dtype: int64

In [7]:
la_crimes['DATE OCC'].str[-8:].value_counts() #selecting only the time and getting the different values of it

DATE OCC
00:00 AM    2117589
Name: count, dtype: int64

* As result, we can delete the data of the time because it does not give additional information to us

In [8]:
la_crimes['Date Rptd'] = la_crimes['Date Rptd'].str[:10] #only keeping the data of the date without the time

In [9]:
la_crimes['DATE OCC'] = la_crimes['DATE OCC'].str[:10] #only keeping the data

* Now we are going to bring the date format to the appropriate one for databases: YYYY-MM-DD

In [10]:
la_crimes['Date Rptd'] = pd.to_datetime(la_crimes['Date Rptd'])
la_crimes['DATE OCC'] = pd.to_datetime(la_crimes['DATE OCC'])
la_crimes['Date Rptd'].sample(5)

2061312   2019-05-01
870394    2014-03-27
1507240   2017-07-01
91540     2010-05-09
743784    2013-05-31
Name: Date Rptd, dtype: datetime64[ns]

## TIME OCC column
* Now we have to fix the column  `TIME OCC` to a '24hour:minutes' format
* First we transform the column to datatype string because it was integer
* Then we add  '00' in front of every date (because the rows with time occured at 12AM have no values for hours
* Lastly we only select the right string characters

In [11]:
la_crimes['TIME OCC'] = la_crimes['TIME OCC'].astype(str)

In [12]:
la_crimes['TIME OCC']= '00'+la_crimes['TIME OCC']

In [13]:
la_crimes['TIME OCC'] = la_crimes['TIME OCC'].str[-4:-2] + ':' +la_crimes['TIME OCC'].str[-2:]

* We join the  columns `DATE OCC` and `TIME OCC` into the column `DATE OCC` so that it will be in the format YYYY-MM-DD hh:mm. It will be saved as datatype datetime YYYY-MM-DD hh:mm:ss in the database
* We drop the column `TIME OCC`

In [14]:
la_crimes['DATE OCC'] = la_crimes['DATE OCC'].astype(str) +' ' + la_crimes['TIME OCC']
la_crimes['DATE OCC'] = pd.to_datetime(la_crimes['DATE OCC'])

In [15]:
la_crimes = la_crimes.drop(['TIME OCC'], axis=1)

## Mocodes column
We see that some crime reports have null values for the column of `Mocodes`
* We replace those values with the value `unknown`

In [16]:
la_crimes.loc[la_crimes['Mocodes'].isna()]

,DR_NO,Date Rptd,DATE OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
15,100100535,2010-01-17,2010-01-16 17:35:00,1,Central,185,2,946,OTHER MISCELLANEOUS CRIME,NaN,...,IC,Invest Cont,946.0,999.0,NaN,NaN,300 E OLYMPIC BL,NaN,34.0389,-118.2550
28,100100578,2010-02-05,2010-02-03 12:55:00,1,Central,185,2,946,OTHER MISCELLANEOUS CRIME,NaN,...,IC,Invest Cont,946.0,999.0,NaN,NaN,1200 MAPLE AV,NaN,34.0357,-118.2563
51,100100654,2010-02-27,2010-02-27 19:55:00,1,Central,174,2,946,OTHER MISCELLANEOUS CRIME,NaN,...,AA,Adult Arrest,946.0,NaN,NaN,NaN,W 7TH ST,S SPRING ST,34.0445,-118.2523
79,100100730,2010-03-23,2010-03-20 12:15:00,1,Central,111,2,647,THROWING OBJECT AT MOVING VEHICLE,NaN,...,IC,Invest Cont,647.0,NaN,NaN,NaN,CESAR E CHAVEZ,FIGUEROA ST,34.0627,-118.2463
102,100100786,2010-04-08,2010-04-08 02:20:00,1,Central,161,1,510,VEHICLE - STOLEN,NaN,...,IC,Invest Cont,510.0,520.0,NaN,NaN,FRANCISCO ST,8TH ST,34.0481,-118.2633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117559,191710450,2019-05-25,2019-05-25 06:30:00,17,Devonshire,1761,1,510,VEHICLE - STOLEN,NaN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,9200 ETON AV,NaN,34.2376,-118.5947
2117561,190306804,2019-02-13,2019-02-10 20:00:00,3,Southwest,335,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),NaN,...,IC,Invest Cont,420.0,NaN,NaN,NaN,2900 8TH AV,NaN,34.0286,-118.3260
2117573,191816545,2019-07-22,2019-07-17 20:30:00,18,Southeast,1836,2,922,CHILD STEALING,NaN,...,AO,Adult Other,922.0,NaN,NaN,NaN,10400 FIRTH AV,NaN,33.9424,-118.2477
2117581,191307168,2019-02-28,2019-02-28 07:00:00,13,Newton,1394,1,510,VEHICLE - STOLEN,NaN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,100 E 67TH ST,NaN,33.9788,-118.2739


In [17]:
indexes_null_mo = la_crimes.loc[la_crimes ['Mocodes'].isna()].index
la_crimes.loc[indexes_null_mo,'Mocodes'] = 'unknown'

## Vict Age column
* Now we are going to fix the column `Vict Age` that don't make any sense
* Specifically we are going to change all negative age numbers to `0` which we consider missing value.

In [18]:
la_crimes['Vict Age'].value_counts()

Vict Age
 0      370038
 25      48145
 26      47508
 27      47061
 24      46798
         ...  
-9           8
-10          1
-11          1
 114         1
 118         1
Name: count, Length: 112, dtype: int64

In [19]:
indexes_neg_age = la_crimes.loc[la_crimes['Vict Age'] < 0]['Vict Age'].index #finding the indexes of negative ages
la_crimes.loc[indexes_neg_age,'Vict Age'] = 0

## Vict Sex column
* Now we fix the column `Vict Sex` which is the victim's sex.
* According to the description the values are: F - Female, M - Male, X - Unknown

In [20]:
la_crimes['Vict Sex'].value_counts()

Vict Sex
M    975294
F    890255
X     55218
H        73
N        17
-         1
Name: count, dtype: int64

* We see there are a bunch of irrelevant values which we change to 'X'

In [21]:
indexes_wrong_sex = la_crimes.loc[(la_crimes['Vict Sex'] != 'M') & (la_crimes['Vict Sex'] != 'F')].index
la_crimes.loc[indexes_wrong_sex,'Vict Sex'] = 'X'

## Vict Descent column
Now we will fix the column `Vict Descent`. We see there are a bunch of nan values. Also, according to the description of the dataset the different descents are: : 
* A - Other Asian 
* B - Black 
* C - Chinese  
* D - Cambodian 
* F - Filipino 
* G - Guamanian 
* H - Hispanic/Latin/Mexican 
* I - American Indian/Alaskan Native 
* J - Japanese 
* K - Korean 
* L - Laotian 
* O - Other 
* P - Pacific Islander 
* S - Samoan 
* U - Hawaiian 
* V - Vietnamese 
* W - White 
* X - Unknown 
* Z - Asian Indian

In [22]:
print("The reported crimes with Nan (null) values for the column Vict Descent are:")
len(la_crimes.loc[la_crimes['Vict Descent'].isna()])

The reported crimes with Nan (null) values for the column Vict Descent are:


196777

In [23]:
la_crimes['Vict Descent'].value_counts()

Vict Descent
H    726529
W    510890
B    335545
O    203204
X     78248
A     51171
K      9178
F      2571
C      1069
I       950
J       419
P       345
V       205
U       191
Z       136
G        85
S        31
D        24
L        18
-         3
Name: count, dtype: int64

In [24]:
indexes_wrong_decent = la_crimes.loc[(la_crimes['Vict Descent'] == '-') | (la_crimes['Vict Descent'].isna())].index
la_crimes.loc[indexes_wrong_decent,'Vict Descent'] = 'X'

## Premis Cd and Premis Desc columns
Now we see that many rows have null values for the column `Premis Cd` 
* Because this column has float values, we replace the null values with `0`

In [81]:
la_crimes.loc[la_crimes['Premis Cd'].isna()].head(10)

,DR_NO,Date Rptd,DATE OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Crm Cd 3,Crm Cd 4,LOCATION,LAT,LON,LOCATION_CD,DATE_RPTD_CD,DATE_OCC_CD,MOCODE_CD,VICTIM_CD


In [26]:
la_crimes.loc[la_crimes['Premis Desc'] =='unknown'][['Premis Cd','Premis Desc']]

,Premis Cd,Premis Desc


In [27]:
indexes_null_premiscd = la_crimes.loc[la_crimes ['Premis Cd'].isna()].index
la_crimes.loc[indexes_null_premiscd,'Premis Cd'] = 0

Now we see that many rows have null values for the column `Premis Desc`

* We replace those null values with `unknown`

In [28]:
la_crimes.loc[la_crimes['Premis Desc'].isna()]

,DR_NO,Date Rptd,DATE OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
6590,100121447,2010-12-12,2010-12-12 11:50:00,1,Central,185,1,110,CRIMINAL HOMICIDE,unknown,...,AA,Adult Arrest,110.0,NaN,NaN,NaN,200 W OLYMPIC BL,NaN,34.0409,-118.2574
32148,100913648,2010-06-21,2010-06-20 14:35:00,9,Van Nuys,915,1,510,VEHICLE - STOLEN,unknown,...,IC,Invest Cont,510.0,NaN,NaN,NaN,7000 VAN NUYS BL,NaN,34.1976,-118.4487
67361,100816222,2010-09-03,2010-04-16 00:01:00,8,West LA,803,2,813,CHILD ANNOYING (17YRS & UNDER),unknown,...,IC,Invest Cont,813.0,NaN,NaN,NaN,2000 MANDAVILLE C,NaN,34.0949,-118.5111
68300,100818222,2010-11-18,2010-10-16 16:00:00,8,West LA,811,2,812,CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 ...,unknown,...,IC,Invest Cont,812.0,NaN,NaN,NaN,1100 LAS PULGAS RD,NaN,34.0528,-118.5393
71555,100908076,2010-03-16,2010-03-15 22:05:00,15,N Hollywood,1547,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),unknown,...,IC,Invest Cont,420.0,NaN,NaN,NaN,BECK,CHANDLER,34.1687,-118.3834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2111857,190223459,2019-12-17,2019-12-15 21:45:00,2,Rampart,236,2,668,"EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)",0344 1309 1822 0913 1803,...,IC,Invest Cont,668.0,NaN,NaN,NaN,100 S ALVARADO ST,NaN,34.0667,-118.2703
2112442,191515929,2019-08-20,2019-08-20 05:40:00,15,N Hollywood,1599,1,310,BURGLARY,2018 0344 1309 1609 1402 1414 0384,...,IC,Invest Cont,310.0,998.0,NaN,NaN,3700 CAHUENGA BL,NaN,34.1350,-118.3612
2116577,190119416,2019-07-28,2019-07-27 18:00:00,1,Central,157,1,320,"BURGLARY, ATTEMPTED",1607,...,IC,Invest Cont,320.0,NaN,NaN,NaN,600 CROCKER ST,NaN,34.0417,-118.2443
2117093,191221441,2019-08-29,2019-08-29 01:00:00,12,77th Street,1268,1,110,CRIMINAL HOMICIDE,1100 0430 1402 1822 0906,...,AA,Adult Arrest,110.0,998.0,NaN,NaN,8100 S BROADWAY,NaN,33.9660,-118.2783


In [29]:
indexes_null_premisdesc = la_crimes.loc[la_crimes ['Premis Desc'].isna()].index
la_crimes.loc[indexes_null_premisdesc,'Premis Desc'] = 'unknown'

Theoretically, for the rows with `Premis Desc` = 'unknown', the column `Premis Cd` should be equal to $0$ (unknown). However, there are some rows with `Premis Cd` different than 0

In [30]:
la_crimes.loc[la_crimes['Premis Desc']=='unknown'][['Premis Desc','Premis Cd']]

,Premis Desc,Premis Cd
6590,unknown,0.0
32148,unknown,0.0
67361,unknown,0.0
68300,unknown,0.0
71555,unknown,0.0
...,...,...
2111857,unknown,418.0
2112442,unknown,256.0
2116577,unknown,256.0
2117093,unknown,256.0


In detail, the rows with `Premis Desc` = 'unknown', have the values of $0$ or $256$ or $418$ or $838$

In [31]:
la_crimes.loc[la_crimes['Premis Desc']=='unknown']['Premis Cd'].unique()

array([  0., 838., 418., 256.])

* Next up we see that all the rows with `Premis Cd` = $418$, have `Premis Desc` = 'unknown`
* Thus we can change `Premis Cd` to $0$ (unknown)

In [32]:
cd418 = la_crimes.loc[la_crimes['Premis Cd']==418][['Premis Desc','Premis Cd']]
cd418['Premis Desc'].unique()

array(['unknown'], dtype=object)

* Next up we see that all the rows with `Premis Cd` = $256$, have `Premis Desc` = 'unknown`
* Thus we can change `Premis Cd` to $0$ (unknown)

In [33]:
cd256 = la_crimes.loc[la_crimes['Premis Cd']== 256][['Premis Desc','Premis Cd']]
cd256['Premis Desc'].unique()

array(['unknown'], dtype=object)

* Next up we see that all the rows with `Premis Cd` = $838$, have `Premis Desc` = 'unknown`
* Thus we can change `Premis Cd` to $0$ (unknown)

In [34]:
cd838 = la_crimes.loc[la_crimes['Premis Cd']== 838][['Premis Desc','Premis Cd']]
cd838['Premis Desc'].unique()

array(['unknown'], dtype=object)

* We change all the rows with `Premis Cd` equal to $256$, $418$ or $838$, to $0$ (unknown)

In [35]:
indexes_wrong_premiscd = la_crimes.loc[(la_crimes ['Premis Cd'] == 256) | 
                                      (la_crimes ['Premis Cd'] == 418) |
                                      (la_crimes ['Premis Cd'] == 838)].index
la_crimes.loc[indexes_wrong_premiscd,'Premis Cd'] = 0

* Lastly, we transform `Premis Cd` from float to integer

In [36]:
la_crimes['Premis Cd'] = la_crimes['Premis Cd'].astype(int)

## Weapon Used Cd and Weapon Desc columns
Some rows have null values for the column `Weapon Used Cd`

* We replace those null values with `0` because this column contains float datatypes

In [37]:
la_crimes.loc[la_crimes['Weapon Used Cd'].isna()]

,DR_NO,Date Rptd,DATE OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,1307355,2010-02-20,2010-02-20 13:50:00,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,...,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,2010-09-13,2010-09-12 00:45:00,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,...,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,2010-08-09,2010-08-09 15:15:00,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,0344,...,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524
5,100100506,2010-01-05,2010-01-04 16:50:00,1,Central,162,1,442,SHOPLIFTING - PETTY THEFT ($950 & UNDER),0344 1402,...,AA,Adult Arrest,442.0,NaN,NaN,NaN,700 W 7TH ST,NaN,34.0480,-118.2577
6,100100508,2010-01-08,2010-01-07 20:05:00,1,Central,182,1,330,BURGLARY FROM VEHICLE,0344,...,IC,Invest Cont,330.0,NaN,NaN,NaN,PICO BL,GRAND AV,34.0389,-118.2643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117577,191408297,2019-03-11,2019-03-08 12:00:00,14,Pacific,1438,1,440,THEFT PLAIN - PETTY ($950 & UNDER),0344 1501 1608 1607 0349,...,IC,Invest Cont,440.0,NaN,NaN,NaN,3600 MIDVALE AV,NaN,34.0204,-118.4139
2117581,191307168,2019-02-28,2019-02-28 07:00:00,13,Newton,1394,1,510,VEHICLE - STOLEN,unknown,...,IC,Invest Cont,510.0,NaN,NaN,NaN,100 E 67TH ST,NaN,33.9788,-118.2739
2117585,190715222,2019-08-15,2019-08-14 18:10:00,7,Wilshire,701,1,331,THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER),1300 0344,...,IC,Invest Cont,331.0,NaN,NaN,NaN,WILLOUGHBY AV,ORLANDO AV,34.0871,-118.3732
2117587,191716777,2019-10-17,2019-10-16 18:00:00,17,Devonshire,1795,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),unknown,...,IC,Invest Cont,420.0,NaN,NaN,NaN,17200 NAPA ST,NaN,34.2266,-118.5085


In [38]:
indexes_null_weaponcd = la_crimes.loc[la_crimes ['Weapon Used Cd'].isna()].index
la_crimes.loc[indexes_null_weaponcd,'Weapon Used Cd'] = 0

Also rows have null values for the column `Weapon Desc`

* We replace those null values with `unknown` because this column contains object datatypes

In [39]:
la_crimes.loc[la_crimes['Weapon Desc'].isna()]

,DR_NO,Date Rptd,DATE OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,1307355,2010-02-20,2010-02-20 13:50:00,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,...,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,2010-09-13,2010-09-12 00:45:00,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,...,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,2010-08-09,2010-08-09 15:15:00,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,0344,...,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524
5,100100506,2010-01-05,2010-01-04 16:50:00,1,Central,162,1,442,SHOPLIFTING - PETTY THEFT ($950 & UNDER),0344 1402,...,AA,Adult Arrest,442.0,NaN,NaN,NaN,700 W 7TH ST,NaN,34.0480,-118.2577
6,100100508,2010-01-08,2010-01-07 20:05:00,1,Central,182,1,330,BURGLARY FROM VEHICLE,0344,...,IC,Invest Cont,330.0,NaN,NaN,NaN,PICO BL,GRAND AV,34.0389,-118.2643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117577,191408297,2019-03-11,2019-03-08 12:00:00,14,Pacific,1438,1,440,THEFT PLAIN - PETTY ($950 & UNDER),0344 1501 1608 1607 0349,...,IC,Invest Cont,440.0,NaN,NaN,NaN,3600 MIDVALE AV,NaN,34.0204,-118.4139
2117581,191307168,2019-02-28,2019-02-28 07:00:00,13,Newton,1394,1,510,VEHICLE - STOLEN,unknown,...,IC,Invest Cont,510.0,NaN,NaN,NaN,100 E 67TH ST,NaN,33.9788,-118.2739
2117585,190715222,2019-08-15,2019-08-14 18:10:00,7,Wilshire,701,1,331,THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER),1300 0344,...,IC,Invest Cont,331.0,NaN,NaN,NaN,WILLOUGHBY AV,ORLANDO AV,34.0871,-118.3732
2117587,191716777,2019-10-17,2019-10-16 18:00:00,17,Devonshire,1795,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),unknown,...,IC,Invest Cont,420.0,NaN,NaN,NaN,17200 NAPA ST,NaN,34.2266,-118.5085


In [40]:
indexes_null_weapondesc = la_crimes.loc[la_crimes ['Weapon Desc'].isna()].index
la_crimes.loc[indexes_null_weapondesc,'Weapon Desc'] = 'unknown'

Lastly, we transform the column `Weapon Used Cd` from float to integer

In [41]:
 la_crimes['Weapon Used Cd'] = la_crimes['Weapon Used Cd'].astype(int)

## Status and Status Desc columns
* Now we check the status of the crime incident
* We see that the columns `Status` and `Status Desc` have some differences.

In [42]:
la_crimes['Status'].value_counts()

Status
IC    1622887
AO     253131
AA     220228
JA      15893
JO       5415
CC         29
TH          1
13          1
19          1
Name: count, dtype: int64

In [43]:
la_crimes['Status Desc'].value_counts()

Status Desc
Invest Cont     1622887
Adult Other      253131
Adult Arrest     220228
Juv Arrest        15893
Juv Other          5415
UNK                  35
Name: count, dtype: int64

* We observe that all the values with a Status `CC` , `19` , `TH` , `13` have a Status Description `UNK`
* That's why we change the rows with a Status `19` , `TH` , `13`(because those only appear once) or null to `CC` 

In [44]:
la_crimes.loc[la_crimes['Status Desc'] == 'UNK']

,DR_NO,Date Rptd,DATE OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
100071,101208618,2010-03-02,2010-03-02 03:30:00,12,77th Street,1248,1,210,ROBBERY,0305 0342 0344 0416 1008,...,NaN,UNK,210.0,NaN,NaN,NaN,70TH ST,MENLO,33.9764,-118.2892
151778,101700682,2010-03-09,2010-03-08 11:55:00,17,Devonshire,1764,2,653,"CREDIT CARDS, FRAUD USE ($950.01 & OVER)",0377 0930 1402 1822,...,CC,UNK,653.0,998.0,NaN,NaN,19300 NORDHOFF ST,NaN,34.2355,-118.5536
160704,101721148,2010-11-15,2010-11-14 17:00:00,17,Devonshire,1756,2,900,VIOLATION OF COURT ORDER,1501,...,CC,UNK,900.0,NaN,NaN,NaN,17800 LASSEN ST,NaN,34.2504,-118.5216
219870,112109831,2011-04-29,2011-04-28 22:00:00,21,Topanga,2139,1,331,THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER),0344 1202,...,CC,UNK,331.0,NaN,NaN,NaN,7300 CORBIN AV,NaN,34.2031,-118.5623
285661,111204921,2011-01-14,2011-01-12 18:30:00,12,77th Street,1266,1,210,ROBBERY,0202 0305 0344 0370 0416 0429 0906 1251 1259 1822,...,CC,UNK,210.0,NaN,NaN,NaN,HOOVER ST,83RD ST,33.9632,-118.2871
400460,111225404,2011-10-26,2011-10-26 07:20:00,12,77th Street,1243,2,920,KIDNAPPING - GRAND ATTEMPT,0305 1251 1258 1313 1822,...,CC,UNK,920.0,NaN,NaN,NaN,68TH ST,VAN NESS,33.9790,-118.3112
486142,141215426,2014-06-30,2012-01-01 12:00:00,12,77th Street,1268,2,354,THEFT OF IDENTITY,0100 1822 0917,...,CC,UNK,354.0,NaN,NaN,NaN,200 E 85TH ST,NaN,33.9610,-118.2717
563438,120619583,2012-07-16,2012-06-16 12:00:00,6,Hollywood,644,2,922,CHILD STEALING,unknown,...,CC,UNK,922.0,986.0,NaN,NaN,1300 N VISTA ST,NaN,34.0944,-118.3517
599080,120123632,2012-11-17,2012-11-16 20:30:00,1,Central,192,2,888,TRESPASSING,0601 1609 0329,...,TH,UNK,888.0,NaN,NaN,NaN,400 W VENICE BL,NaN,34.0365,-118.2676
631717,131411831,2013-04-26,2013-04-02 23:00:00,14,Pacific,1494,1,440,THEFT PLAIN - PETTY ($950 & UNDER),0344,...,CC,UNK,440.0,NaN,NaN,NaN,00 WORLD WY,NaN,33.9454,-118.3998


In [45]:
indexes_status = la_crimes.loc[(la_crimes['Status'] == '19')|
                              (la_crimes['Status'] == '13') | (la_crimes['Status'] == 'TH')|
                               (la_crimes['Status'].isna())
                              ]['Status'].index #finding the indexes of nan Status or status 19,13,TH
la_crimes.loc[indexes_status,'Status'] = 'CC'

* We consider that the value `UNK` for the column `Status Desc` means unknown
* Then we will change the Nan values for column `Status Desc` to `UNK`

In [46]:
indexes_status_desc = la_crimes.loc[la_crimes['Status Desc'].isna()]['Status'].index #finding the indexes of nan Status or status 19,13,TH
la_crimes.loc[indexes_status_desc,'Status Desc'] = 'UNK'

## Crm Cd columns
Also, reading the documentation of the columns `Crm Cd` indicates the crime committed. `Crm Cd 1` is the primary and most serious one. Crime Code 2, 3, and 4 are respectively less serious offenses
* As a result it doesn't make sense to only have a `Crm Cd 2` offense with no `Crm Cd 1` offense
* Thus we will update all crime reports with null `Crm Cd 1` and some `Crm Cd 2` or `Crm Cd 3` offenses so that in every case `Crm Cd 1` has a value.
* If there were 2 offenses then there will be only values for `Crm Cd 1` and `Crm Cd 2` and so on

In [47]:
indexes_crm1null = la_crimes.loc[(la_crimes ['Crm Cd 1'].isna()) ].index
la_crimes.loc[indexes_crm1null,:]

,DR_NO,Date Rptd,DATE OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
55526,100707214,2010-03-14,2010-03-13 02:30:00,7,Wilshire,767,2,624,BATTERY - SIMPLE ASSAULT,0344 0400 0416 1300,...,IC,Invest Cont,NaN,624.0,NaN,NaN,PICO BL,NORTON AV,34.0476,-118.3239
288372,110310134,2011-04-08,2011-03-26 08:00:00,3,Southwest,312,2,942,BRIBERY,1300 1402,...,IC,Invest Cont,NaN,942.0,99.0,NaN,5100 ROSELAND ST,NaN,34.0274,-118.3542
358671,110811926,2011-07-01,2011-07-01 20:11:00,8,West LA,835,1,210,ROBBERY,unknown,...,IC,Invest Cont,NaN,210.0,NaN,NaN,11000 SANTA MONICA BL,NaN,34.0484,-118.4411
507922,120325216,2012-11-19,2012-11-19 19:30:00,3,Southwest,329,1,440,THEFT PLAIN - PETTY ($950 & UNDER),0344,...,IC,Invest Cont,NaN,440.0,NaN,NaN,500 W 27TH ST,NaN,34.0268,-118.2753
1176957,150318476,2015-08-17,2015-08-16 12:00:00,3,Southwest,363,1,761,BRANDISH WEAPON,0913 0906 0334 0421 0319 0444 0432 1816,...,AO,Adult Other,NaN,761.0,93.0,NaN,4100 PALMWOOD DR,NaN,34.0137,-118.3435
1189061,150517852,2015-11-09,2015-10-09 18:00:00,5,Harbor,529,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),0344 1300 1606 0321,...,IC,Invest Cont,NaN,420.0,NaN,NaN,200 BERTH,NaN,33.7753,-118.2456
1699079,181824031,2018-12-13,2018-12-13 19:40:00,18,Southeast,1842,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",0906 0450 1402,...,IC,Invest Cont,NaN,230.0,NaN,NaN,400 W 109TH ST,NaN,33.9374,-118.2805
1877245,181117551,2018-10-05,2018-10-05 09:00:00,11,Northeast,1162,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",0445 0432 1822 0443 1266,...,AA,Adult Arrest,NaN,230.0,93.0,NaN,1300 N VERMONT AV,NaN,34.0956,-118.2918
1980173,191400827,2019-07-20,2019-07-20 09:45:00,14,Pacific,1463,2,888,TRESPASSING,1501,...,IC,Invest Cont,NaN,888.0,NaN,NaN,5300 ALLA RD,NaN,33.9779,-118.4264
2083390,190308122,2019-03-05,2019-03-05 21:30:00,3,Southwest,395,1,331,THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER),0216 0344 1606 1822 1300,...,IC,Invest Cont,NaN,331.0,NaN,NaN,39TH ST,NORMANDIE AV,34.0073,-118.3108


In [48]:
la_crimes.loc[indexes_crm1null,'Crm Cd 1'] = la_crimes.loc[indexes_crm1null,'Crm Cd 2']
la_crimes.loc[indexes_crm1null,'Crm Cd 2'] = la_crimes.loc[indexes_crm1null,'Crm Cd 3']
la_crimes.loc[indexes_crm1null,'Crm Cd 3'] = la_crimes.loc[indexes_crm1null,'Crm Cd 4']

We fill up all the null values for `Crm Cd 2` and `Crm Cd 3` and `Crm Cd 4` with -1 (meaning none is found) because those columns consist of float datatype

In [49]:
la_crimes ['Crm Cd 2'] = la_crimes ['Crm Cd 2'].fillna(-1)
la_crimes ['Crm Cd 3'] = la_crimes ['Crm Cd 3'].fillna(-1)
la_crimes ['Crm Cd 4'] = la_crimes ['Crm Cd 4'].fillna(-1)

Lastly, we transform the columns `Crm Cd 1`, `Crm Cd 2`, `Crm Cd 3`, `Crm Cd 4` from float to integer

In [50]:
la_crimes['Crm Cd 1'] = la_crimes['Crm Cd 1'].astype(int)
la_crimes['Crm Cd 2'] = la_crimes['Crm Cd 2'].astype(int)
la_crimes['Crm Cd 3'] = la_crimes['Crm Cd 3'].astype(int)
la_crimes['Crm Cd 4'] = la_crimes['Crm Cd 4'].astype(int)

* We check if for every crime incident, the column `Crm Cd` and `Crm Cd 1` are the same.
* Some are not the same so we consider them mistaken because `Crm Cd 1` is the primary crime commited and `Crm Cd` describes the crime commited
* For those incidents we set `Crm Cd 1` to be equal to `Crm Cd`

* First case is that `Crm Cd` has the same value as `Crm Cd 2`
* Then we just have to swap `Crm Cd 1` and `Crm Cd 2`

In [51]:
indexes_crm_crm2 = la_crimes.loc[la_crimes ['Crm Cd'] ==  la_crimes ['Crm Cd 2']].index
la_crimes.loc[indexes_crm_crm2,'Crm Cd 2'] = la_crimes.loc[indexes_crm_crm2,'Crm Cd 1'].copy()
la_crimes.loc[indexes_crm_crm2,'Crm Cd 1'] = la_crimes.loc[indexes_crm_crm2,'Crm Cd'].copy()

* Second case is that `Crm Cd` has the same value as `Crm Cd 3`
* Then we just have to swap `Crm Cd 1` and `Crm Cd 3`

In [52]:
indexes_crm_crm3 = la_crimes.loc[la_crimes ['Crm Cd'] ==  la_crimes ['Crm Cd 3']].index
la_crimes.loc[indexes_crm_crm3,'Crm Cd 3'] = la_crimes.loc[indexes_crm_crm3,'Crm Cd 1'].copy()
la_crimes.loc[indexes_crm_crm3,'Crm Cd 1'] = la_crimes.loc[indexes_crm_crm3,'Crm Cd'].copy()

* Third case is that `Crm Cd` has the same value as `Crm Cd 4`
* However, there is no incident like that

In [53]:
la_crimes.loc[la_crimes ['Crm Cd'] ==  la_crimes ['Crm Cd 4']].index

Index([], dtype='int64')

* Lastly, because `Crm Cd` and `Crm Cd 1` are the same now, we can drop `Crm Cd 1`

In [54]:
la_crimes = la_crimes.drop(['Crm Cd 1'], axis=1)

## LOCATION Column
* Some locations have a bunch of white spaces in between their words

In [55]:
la_crimes ['LOCATION'].value_counts()

LOCATION
6TH                          ST            4757
7TH                          ST            3775
9300    TAMPA                        AV    3660
6TH                                        3235
6600    TOPANGA CANYON               BL    3066
                                           ... 
6900    WOODROW WILSON                        1
4500    LYNNFIELD                    ST       1
1700    HOLLYWOOD                    BL       1
6000    GUTHRIE                      AV       1
4500    ALAMEDA                               1
Name: count, Length: 75477, dtype: int64

* The unnecessary white spaces are removed from the middle, front and back of the words

In [56]:
 la_crimes['LOCATION'] = la_crimes['LOCATION'].str.replace(' +', ' ').str.strip()

## Cross Street column
* Moreover, we see that there are many null values for the column `Cross Street`
* We changed those to 'unknown'

In [57]:
indexes_null_cross = la_crimes.loc[la_crimes ['Cross Street'].isna()].index
la_crimes.loc[indexes_null_cross,'Cross Street'] = 'unknown'

* In addition, we see many cross streets have a bunch of white spaces in between the name and the street (AV/BL).

In [58]:
la_crimes['Cross Street'].value_counts()

Cross Street
unknown                            1762093
BROADWAY                              6158
FIGUEROA                              3802
VERMONT                      AV       3745
SAN PEDRO                             3661
                                    ...   
PRIMERA                                  1
MUSKINGUM                                1
MARQUEZ                      DR          1
N  VISTA                                 1
EDGEWATER                    AV          1
Name: count, Length: 12875, dtype: int64

* We are going to remove the unnecessary white spaces in the middle, front and back of the words 

In [59]:
la_crimes['Cross Street'] = la_crimes['Cross Street'].str.replace(' +', ' ').str.strip()

* Now we observe that for most crime incidents this column has the value `unknown`
* In detail that is true, for the $1.76$ million incidents out of the $2.1$ million incidents that there are
* Thus we decide to drop this column

In [60]:
la_crimes.loc[la_crimes['Cross Street'] == 'unknown']

,DR_NO,Date Rptd,DATE OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Weapon Desc,Status,Status Desc,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,1307355,2010-02-20,2010-02-20 13:50:00,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,...,unknown,AA,Adult Arrest,-1,-1,-1,300 E GAGE AV,unknown,33.9825,-118.2695
2,70309629,2010-08-09,2010-08-09 15:15:00,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,0344,...,unknown,IC,Invest Cont,-1,-1,-1,1300 E 21ST ST,unknown,34.0224,-118.2524
5,100100506,2010-01-05,2010-01-04 16:50:00,1,Central,162,1,442,SHOPLIFTING - PETTY THEFT ($950 & UNDER),0344 1402,...,unknown,AA,Adult Arrest,-1,-1,-1,700 W 7TH ST,unknown,34.0480,-118.2577
7,100100509,2010-01-09,2010-01-08 21:00:00,1,Central,157,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",0416,...,UNKNOWN WEAPON/OTHER WEAPON,AA,Adult Arrest,-1,-1,-1,500 CROCKER ST,unknown,34.0435,-118.2427
8,100100510,2010-01-09,2010-01-09 02:30:00,1,Central,171,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",0400 0416,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,-1,-1,-1,800 W OLYMPIC BL,unknown,34.0450,-118.2640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117582,190906699,2019-02-24,2019-02-23 22:20:00,9,Van Nuys,904,1,210,ROBBERY,0344 0302 0334 0355 1310 1420 1822 0354,...,OTHER FIREARM,IC,Invest Cont,998,-1,-1,7600 WILLIS AV,unknown,34.2085,-118.4553
2117583,190506304,2019-02-22,2019-02-22 08:40:00,5,Harbor,569,2,627,CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT,0443 0419 0416 1259,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AO,Adult Other,-1,-1,-1,100 W 22ND ST,unknown,33.7257,-118.2801
2117584,190608903,2019-03-28,2019-03-28 04:00:00,6,Hollywood,644,1,648,ARSON,0601 1501,...,FIRE,IC,Invest Cont,-1,-1,-1,1400 N LA BREA AV,unknown,34.0962,-118.3490
2117587,191716777,2019-10-17,2019-10-16 18:00:00,17,Devonshire,1795,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),unknown,...,unknown,IC,Invest Cont,-1,-1,-1,17200 NAPA ST,unknown,34.2266,-118.5085


In [61]:
la_crimes = la_crimes.drop(['Cross Street'], axis=1)

## LONGTITUDE-LATITUDE
* We see that some crimes have values for longtitude and latitude equal to $0$
* Those values are considered missing because they correspond to the Gulf of Guinea.

In [62]:
la_crimes.loc[(la_crimes['LON'] == 0) & (la_crimes['LAT'] == 0) ]

,DR_NO,Date Rptd,DATE OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,LAT,LON
49699,100618355,2010-07-14,2010-07-12 19:00:00,6,Hollywood,665,1,330,BURGLARY FROM VEHICLE,0344 1300 1302,...,0,unknown,IC,Invest Cont,-1,-1,-1,900 N CISTRUS AV,0.0,0.0
49795,100618603,2010-07-19,2010-07-19 23:45:00,6,Hollywood,665,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329 0906,...,0,unknown,AA,Adult Arrest,998,-1,-1,6300 WILLOUGBY,0.0,0.0
60848,100718479,2010-11-29,2010-11-29 16:30:00,7,Wilshire,709,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",0416 1402,...,122,HECKLER & KOCH 93 SEMIAUTOMATIC ASSAULT RIFLE,IC,Invest Cont,998,-1,-1,HARBOR,0.0,0.0
85012,101016365,2010-09-09,2010-08-23 15:00:00,10,West Valley,1000,2,626,INTIMATE PARTNER - SIMPLE ASSAULT,0416 2000,...,0,unknown,IC,Invest Cont,-1,-1,-1,CITY OF WINNETKA,0.0,0.0
123980,101409719,2010-04-01,2010-03-30 21:00:00,14,Pacific,1412,1,510,VEHICLE - STOLEN,unknown,...,0,unknown,IC,Invest Cont,-1,-1,-1,WINDWARD AV,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979527,212010145,2021-06-07,2019-06-20 12:00:00,20,Olympic,2074,2,812,CRM AGNST CHLD (13 OR UNDER) (14-15 & SUSP 10 ...,0509 0913 0522 0510,...,400,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,860,-1,-1,1800 ROOSEVELT AV,0.0,0.0
1984638,191915311,2019-08-22,2019-08-21 11:00:00,19,Mission,1900,1,236,INTIMATE PARTNER - AGGRAVATED ASSAULT,0400 0448 2000 0913,...,400,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AA,Adult Arrest,-1,-1,-1,UNKNOWN,0.0,0.0
1998038,191409480,2019-03-31,2019-03-31 03:00:00,14,Pacific,1400,2,624,BATTERY - SIMPLE ASSAULT,0400 0416,...,400,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,-1,-1,-1,96TH,0.0,0.0
2016324,190918499,2019-10-18,2019-10-15 16:30:00,9,Van Nuys,936,2,626,INTIMATE PARTNER - SIMPLE ASSAULT,1814 2000 0448,...,400,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AA,Adult Arrest,-1,-1,-1,13600 LANAY ST,0.0,0.0


# SAVING THE CLEANED DATASET
* We save the cleaned dataset as a new csv file called `LA Crime_Cleared.csv`, since we are going to use it for analysis purposes after

In [63]:
la_crimes.to_csv('LA Crime_Cleared.csv', index=False)

# COLUMNS SEPARATING + DIMENSION TABLES

 We will organize the database based on a star schema. In the middle the fact table will contain the crime incidents. In addition, there will be several different dimensions which will relate to each particular incident. Specifically the dimensions will be the location, the date reported,the date occured, the suspect, the victim, the crime commited, the premises (meaning the type of structure, vehicle, or location where the crime took place), the weapon used, the status of the crime and the coordinate dimension

## Location dimension
* We start by creating the dimension of the location which will include the location, the district, the area code and the area name 
* Particularly we take the columns `LOCATION` `Rpt Dist No` and `AREA NAME` and clear the duplicate values
* The results are saved in a new dataframe called `location_dimension`

In [64]:
la_crimes[['LOCATION','Rpt Dist No','AREA','AREA NAME']].drop_duplicates()

,LOCATION,Rpt Dist No,AREA,AREA NAME
0,300 E GAGE AV,1385,13,Newton
1,SEPULVEDA BL,1485,14,Pacific
2,1300 E 21ST ST,1324,13,Newton
3,CAHUENGA BL,646,6,Hollywood
4,8TH ST,176,1,Central
...,...,...,...,...
2117336,8300 81ST ST,1483,14,Pacific
2117352,4500 ALAMEDA,1367,13,Newton
2117426,WASHINGTON,2069,20,Olympic
2117483,CRYSTAL,1133,11,Northeast


* We see that each district number (`Rpt Dist No`) corresponds to one area name
* That means if we just know the district where a crime happened, we can find out the area where it took place because every district belongs to an area
* However, each location can correspond to different districts and areas because of the likelihood of using the same names

In [65]:
la_crimes[['Rpt Dist No','AREA NAME']].drop_duplicates()

,Rpt Dist No,AREA NAME
0,1385,Newton
1,1485,Pacific
2,1324,Newton
3,646,Hollywood
4,176,Central
...,...,...
1969369,1086,West Valley
2059941,1686,Foothill
2080699,1784,Devonshire
2092560,1647,Foothill


In [66]:
location_dimension = la_crimes[['LOCATION','Rpt Dist No','AREA NAME']].drop_duplicates()
location_dimension = location_dimension.reset_index(drop = True)
location_dimension['LOCATION_CD'] = location_dimension.index
location_dimension

,LOCATION,Rpt Dist No,AREA NAME,LOCATION_CD
0,300 E GAGE AV,1385,Newton,0
1,SEPULVEDA BL,1485,Pacific,1
2,1300 E 21ST ST,1324,Newton,2
3,CAHUENGA BL,646,Hollywood,3
4,8TH ST,176,Central,4
...,...,...,...,...
124124,8300 81ST ST,1483,Pacific,124124
124125,4500 ALAMEDA,1367,Newton,124125
124126,WASHINGTON,2069,Olympic,124126
124127,CRYSTAL,1133,Northeast,124127


* We will use the index column of the new dataframe `location_dimension` as a foreign key and thus we need to add a new column to `la_crimes` containing for each incident the value of this foreign key
* After we do that, the columns `LOCATION` `Rpt Dist No` `ΑREA` and `AREA NAME` are no longer needed in the dataframe `la_crimes` as we have the mapping from the column `LOCATION_CD` to the dimension table `location_dimension`

In [67]:
la_crimes = pd.merge(la_crimes, 
               location_dimension, 
               on=['LOCATION','Rpt Dist No','AREA NAME'],
               how='inner')
la_crimes

,DR_NO,Date Rptd,DATE OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Weapon Desc,Status,Status Desc,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,LAT,LON,LOCATION_CD
0,1307355,2010-02-20,2010-02-20 13:50:00,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,...,unknown,AA,Adult Arrest,-1,-1,-1,300 E GAGE AV,33.9825,-118.2695,0
1,101304962,2010-01-14,2010-01-14 20:45:00,13,Newton,1385,2,946,OTHER MISCELLANEOUS CRIME,0421 0906,...,unknown,AO,Adult Other,-1,-1,-1,300 E GAGE AV,33.9825,-118.2695,0
2,101304963,2010-01-14,2010-01-14 20:45:00,13,Newton,1385,2,946,OTHER MISCELLANEOUS CRIME,0421,...,VERBAL THREAT,IC,Invest Cont,-1,-1,-1,300 E GAGE AV,33.9825,-118.2695,0
3,101305011,2010-01-15,2010-01-15 16:15:00,13,Newton,1385,2,946,OTHER MISCELLANEOUS CRIME,0334 0432 0913,...,unknown,AO,Adult Other,-1,-1,-1,300 E GAGE AV,33.9825,-118.2695,0
4,101307014,2010-02-16,2010-02-16 21:00:00,13,Newton,1385,1,331,THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER),0344 1606,...,unknown,IC,Invest Cont,-1,-1,-1,300 E GAGE AV,33.9825,-118.2695,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117584,191416200,2019-07-16,2019-06-24 12:00:00,14,Pacific,1483,2,354,THEFT OF IDENTITY,1822,...,unknown,IC,Invest Cont,-1,-1,-1,8300 81ST ST,33.9568,-118.4290,124124
2117585,191308117,2019-03-16,2019-03-16 14:30:00,13,Newton,1367,1,350,"THEFT, PERSON",0344 0602 0554 0907 1251 1259 0416,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,-1,-1,-1,4500 ALAMEDA,33.9987,-118.2388,124125
2117586,192019775,2019-11-09,2019-11-09 23:00:00,20,Olympic,2069,2,946,OTHER MISCELLANEOUS CRIME,1300 1501,...,unknown,IC,Invest Cont,-1,-1,-1,WASHINGTON,34.0499,-118.2879,124126
2117587,191114224,2019-07-27,2019-07-22 04:30:00,11,Northeast,1133,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",0416 0400,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,-1,-1,-1,CRYSTAL,34.1119,-118.2529,124127


## Date reported dimension
* Next up we create the dimension of the date that the crime was reported
* Particularly we take the column `Date Rptd` and clear the duplicate values
* The results are saved in a new dataframe called `date_rptd_dimension`
* We also save the index of the new dataframe as `DATE_RPTD_CD`

In [68]:
date_rptd_dimension = la_crimes['Date Rptd'].drop_duplicates()
date_rptd_dimension = pd.DataFrame({'Date Rptd':date_rptd_dimension})
date_rptd_dimension = date_rptd_dimension.reset_index(drop = True)
date_rptd_dimension['DATE_RPTD_CD'] = date_rptd_dimension.index
date_rptd_dimension

,Date Rptd,DATE_RPTD_CD
0,2010-02-20,0
1,2010-01-14,1
2,2010-01-15,2
3,2010-02-16,3
4,2010-04-03,4
...,...,...
4164,2021-03-07,4164
4165,2020-07-04,4165
4166,2021-01-31,4166
4167,2021-05-08,4167


* We will use the index column of the new dataframe `date_rptd_dimension` as a foreign key and thus we need to add a new column to `la_crimes` containing for each incident the value of this foreign key
* After we do that, the column `Date Rptd` is no longer needed in the dataframe `la_crimes` as we have the mapping from the column `DATE_RPTD_CD` to the dimension table `date_rptd_dimension`

In [69]:
la_crimes = pd.merge(la_crimes, 
               date_rptd_dimension, 
               on='Date Rptd',
               how='inner')
la_crimes

,DR_NO,Date Rptd,DATE OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Status,Status Desc,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,LAT,LON,LOCATION_CD,DATE_RPTD_CD
0,1307355,2010-02-20,2010-02-20 13:50:00,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,...,AA,Adult Arrest,-1,-1,-1,300 E GAGE AV,33.9825,-118.2695,0,0
1,101307354,2010-02-20,2010-02-20 14:35:00,13,Newton,1385,2,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,0371 0421 0906,...,AA,Adult Arrest,-1,-1,-1,300 E GAGE AV,33.9825,-118.2695,0,0
2,101307355,2010-02-20,2010-02-20 13:50:00,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,...,IC,Invest Cont,-1,-1,-1,300 E GAGE AV,33.9825,-118.2695,0,0
3,100106680,2010-02-20,2010-02-20 17:00:00,1,Central,152,2,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,0421 0903,...,AA,Adult Arrest,-1,-1,-1,600 W 5TH ST,34.0502,-118.2540,13,0
4,100106675,2010-02-20,2010-02-20 11:00:00,1,Central,192,2,626,INTIMATE PARTNER - SIMPLE ASSAULT,0416 1241 2000,...,AO,Adult Other,-1,-1,-1,1300 S FLOWER ST,34.0401,-118.2668,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117584,200612163,2020-07-04,2010-07-02 15:45:00,6,Hollywood,663,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0400 1822 0447,...,IC,Invest Cont,-1,-1,-1,7000 SANTA MONICA BL,34.0907,-118.3421,59241,4165
2117585,211705349,2021-01-31,2019-10-31 12:05:00,17,Devonshire,1777,2,354,THEFT OF IDENTITY,0928,...,IC,Invest Cont,-1,-1,-1,9200 HAYVENHURST AV,34.2376,-118.4938,61052,4166
2117586,211004978,2021-01-31,2015-01-20 00:01:00,10,West Valley,1023,1,121,"RAPE, FORCIBLE",0500 1258 0501 0516 0522 0532 0400 0913 0558,...,AA,Adult Arrest,812,-1,-1,6900 TAMPA AV,34.1960,-118.5535,71774,4166
2117587,210408422,2021-05-08,2019-11-01 12:00:00,4,Hollenbeck,421,2,354,THEFT OF IDENTITY,0100,...,IC,Invest Cont,-1,-1,-1,1700 MOZART ST,34.0674,-118.2223,63404,4167


## Date occurred dimension
* Next up we create the dimension of the date that the crime happened
* Particularly we take the column `DATE OCC` and clear the duplicate values
* The results are saved in a new dataframe called `date_occ_dimension`
* We also save the index of the new dataframe as `DATE_OCC_CD`

In [70]:
date_occ_dimension = la_crimes['DATE OCC'].drop_duplicates()
date_occ_dimension = pd.DataFrame({'DATE OCC':date_occ_dimension})
date_occ_dimension = date_occ_dimension.reset_index(drop = True)
date_occ_dimension['DATE_OCC_CD'] = date_occ_dimension.index
date_occ_dimension

,DATE OCC,DATE_OCC_CD
0,2010-02-20 13:50:00,0
1,2010-02-20 14:35:00,1
2,2010-02-20 17:00:00,2
3,2010-02-20 11:00:00,3
4,2010-02-16 08:15:00,4
...,...,...
626415,2019-07-21 08:26:00,626415
626416,2016-03-02 12:05:00,626416
626417,2014-11-01 10:20:00,626417
626418,2019-01-29 13:15:00,626418


* We will use the index column of the new dataframe `date_occ_dimension` as a foreign key and thus we need to add a new column to `la_crimes` containing for each incident the value of this foreign key
* After we do that, the column `DATE OCC` is no longer needed in the dataframe `la_crimes` as we have the mapping from the column `DATE_OCC_CD` to the dimension table `date_occ_dimension`

In [71]:
la_crimes = pd.merge(la_crimes, 
               date_occ_dimension, 
               on='DATE OCC',
               how='inner')
la_crimes

,DR_NO,Date Rptd,DATE OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Status Desc,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,LAT,LON,LOCATION_CD,DATE_RPTD_CD,DATE_OCC_CD
0,1307355,2010-02-20,2010-02-20 13:50:00,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,...,Adult Arrest,-1,-1,-1,300 E GAGE AV,33.9825,-118.2695,0,0,0
1,101307355,2010-02-20,2010-02-20 13:50:00,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,...,Invest Cont,-1,-1,-1,300 E GAGE AV,33.9825,-118.2695,0,0,0
2,100906831,2010-02-20,2010-02-20 13:50:00,9,Van Nuys,943,2,928,THREATENING PHONE CALLS/LETTERS,0421,...,Invest Cont,-1,-1,-1,14100 OXNARD ST,34.1794,-118.4400,20627,0,0
3,101307354,2010-02-20,2010-02-20 14:35:00,13,Newton,1385,2,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,0371 0421 0906,...,Adult Arrest,-1,-1,-1,300 E GAGE AV,33.9825,-118.2695,0,0,1
4,100106680,2010-02-20,2010-02-20 17:00:00,1,Central,152,2,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,0421 0903,...,Adult Arrest,-1,-1,-1,600 W 5TH ST,34.0502,-118.2540,13,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117584,201411978,2020-06-07,2019-07-21 08:26:00,14,Pacific,1465,2,956,"LETTERS, LEWD - TELEPHONE CALLS, LEWD",1501,...,Adult Other,-1,-1,-1,12000 JUNIETTE ST,33.9841,-118.4064,37857,4155,626415
2117585,202018908,2020-12-26,2016-03-02 12:05:00,20,Olympic,2044,2,354,THEFT OF IDENTITY,0930 0928 1822 0100,...,Invest Cont,-1,-1,-1,900 S ARDMORE AV,34.0545,-118.3020,54831,4159,626416
2117586,201821309,2020-11-26,2014-11-01 10:20:00,18,Southeast,1837,2,354,THEFT OF IDENTITY,0100 1822 0120,...,Invest Cont,-1,-1,-1,1600 E 108TH ST,33.9378,-118.2463,48507,4161,626417
2117587,201821746,2020-12-06,2019-01-29 13:15:00,18,Southeast,1827,2,354,THEFT OF IDENTITY,0100 1822 0930 0935 0925,...,Invest Cont,-1,-1,-1,9500 GRAPE ST,33.9482,-118.2363,48340,4162,626418


## Suspect dimension 
* Now we create the dimension of the suspect. Mocodes are activities associated with the suspect in commission of the crime
* Particularly we take the column `Mocodes` and clear the duplicate values
* The results are saved in a new dataframe called `suspect_dimension`
* We also save the index of the new dataframe as `MOCODES_CD`

In [72]:
suspect_dimension = la_crimes['Mocodes'].drop_duplicates()
suspect_dimension = pd.DataFrame({'Mocodes':suspect_dimension})
suspect_dimension = suspect_dimension.reset_index(drop = True)
suspect_dimension['MOCODE_CD'] = suspect_dimension.index
suspect_dimension

,Mocodes,MOCODE_CD
0,0913 1814 2000,0
1,0421,1
2,0371 0421 0906,2
3,0421 0903,3
4,0344,4
...,...,...
485222,0923 0930 0928 0929,485222
485223,1259 0400 0500 1822 1251 0444 0448,485223
485224,0913 0400 0421 1813 2000,485224
485225,1202 0445 0447 1266 0400 2004 1822,485225


* We will use the index column of the new dataframe `suspect_dimension` as a foreign key and thus we need to add a new column to `la_crimes` containing for each incident the value of this foreign key
* After we do that, the column `Mocodes` is no longer needed in the dataframe `la_crimes` as we have the mapping from the column `MOCODE_CD` to the dimension table `suspect_dimension`

In [73]:
la_crimes = pd.merge(la_crimes, 
               suspect_dimension, 
               on='Mocodes',
               how='inner')
la_crimes

,DR_NO,Date Rptd,DATE OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,LAT,LON,LOCATION_CD,DATE_RPTD_CD,DATE_OCC_CD,MOCODE_CD
0,1307355,2010-02-20,2010-02-20 13:50:00,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,...,-1,-1,-1,300 E GAGE AV,33.9825,-118.2695,0,0,0,0
1,101307355,2010-02-20,2010-02-20 13:50:00,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,...,-1,-1,-1,300 E GAGE AV,33.9825,-118.2695,0,0,0,0
2,100806537,2010-02-28,2010-02-20 14:45:00,8,West LA,885,2,624,BATTERY - SIMPLE ASSAULT,0913 1814 2000,...,-1,-1,-1,2900 OVERLAND AV,34.0340,-118.4188,18032,3475,103,0
3,101300529,2010-01-15,2010-01-15 22:00:00,13,Newton,1371,1,310,BURGLARY,0913 1814 2000,...,-1,-1,-1,100 W 57TH ST,33.9906,-118.2739,31893,2,720,0
4,111314249,2011-06-10,2010-09-01 08:00:00,13,Newton,1372,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,...,-1,-1,-1,300 E 56TH ST,33.9916,-118.2701,31872,2160,3830,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117584,210908400,2021-04-21,2017-04-01 12:35:00,9,Van Nuys,937,2,354,THEFT OF IDENTITY,0923 0930 0928 0929,...,-1,-1,-1,6300 COSTELLO AV,34.1849,-118.4378,20519,4123,626398,485222
2117585,201715086,2020-11-04,2019-10-12 00:17:00,17,Devonshire,1728,1,121,"RAPE, FORCIBLE",1259 0400 0500 1822 1251 0444 0448,...,-1,-1,-1,PORTER RANCH,34.2707,-118.5345,123608,4126,626403,485223
2117586,201223258,2020-10-17,2010-10-17 22:40:00,12,77th Street,1245,2,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,0913 0400 0421 1813 2000,...,-1,-1,-1,6400 S VERMONT AV,33.9808,-118.2917,29360,4135,626405,485224
2117587,201716266,2020-12-15,2019-12-20 19:25:00,17,Devonshire,1757,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",1202 0445 0447 1266 0400 2004 1822,...,-1,-1,-1,10100 BALBOA BL,34.2539,-118.4967,44424,4139,626406,485225


## Victim dimension
* Next up the dimension of the vicitm will be made
* Particularly we take the columns `Vict Sex` and `Vict Descent`  and clear the duplicate values
* The results are saved in a new dataframe called `victim_dimension`

In [74]:
victim_dimension = la_crimes[['Vict Sex','Vict Descent']].drop_duplicates()
victim_dimension = victim_dimension.reset_index(drop=True)
victim_dimension['VICTIM_CD'] = victim_dimension.index
victim_dimension

,Vict Sex,Vict Descent,VICTIM_CD
0,M,H,0
1,F,B,1
2,M,B,2
3,F,H,3
4,M,W,4
5,F,W,5
6,F,O,6
7,M,O,7
8,M,K,8
9,F,A,9


* We will use the index column of the new dataframe `victim_dimension` as a foreign key and thus we need to add a new column to `la_crimes` containing for each incident the value of this foreign key
* After we do that, the columns `Vict Sex` `Vict Descent`is no longer needed in the dataframe `la_crimes` as we have the mapping from the column `VICTIM_CD` to the dimension table `victim_dimension`

In [75]:
la_crimes = pd.merge(la_crimes, 
               victim_dimension, 
               on=['Vict Sex','Vict Descent'],
               how='inner')
la_crimes

,DR_NO,Date Rptd,DATE OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Crm Cd 3,Crm Cd 4,LOCATION,LAT,LON,LOCATION_CD,DATE_RPTD_CD,DATE_OCC_CD,MOCODE_CD,VICTIM_CD
0,1307355,2010-02-20,2010-02-20 13:50:00,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,...,-1,-1,300 E GAGE AV,33.9825,-118.2695,0,0,0,0,0
1,111315925,2011-07-03,2011-06-27 13:00:00,13,Newton,1352,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,...,-1,-1,400 E 47TH ST,34.0007,-118.2696,32637,2601,171068,0,0
2,152014532,2015-08-10,2015-08-07 15:30:00,20,Olympic,2069,2,901,VIOLATION OF RESTRAINING ORDER,0913 1814 2000,...,-1,-1,1100 S VERMONT AV,34.0509,-118.2916,53380,1905,211794,0,0
3,152010566,2015-05-15,2015-05-15 23:00:00,20,Olympic,2069,2,901,VIOLATION OF RESTRAINING ORDER,0913 1814 2000,...,-1,-1,1100 S WESTMORELAND AV,34.0508,-118.2891,53909,2295,216664,0,0
4,100906832,2010-02-20,2010-02-20 14:00:00,9,Van Nuys,943,2,928,THREATENING PHONE CALLS/LETTERS,0421,...,-1,-1,14100 OXNARD ST,34.1794,-118.4400,20627,0,12,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117584,162008634,2016-04-05,2016-04-04 18:00:00,20,Olympic,2041,1,330,BURGLARY FROM VEHICLE,1612 0358 0344,...,-1,-1,900 3RD AV,34.0559,-118.3165,53680,1950,88506,57439,46
2117585,171119454,2017-11-14,2017-11-12 11:00:00,11,Northeast,1102,2,624,BATTERY - SIMPLE ASSAULT,0416 0432 1202 0448 0913,...,-1,-1,5000 EDENHURST AV,34.1434,-118.2732,94955,1173,68945,111544,46
2117586,180320562,2018-09-06,2018-09-06 07:00:00,3,Southwest,325,1,440,THEFT PLAIN - PETTY ($950 & UNDER),0344 0354,...,-1,-1,2600 S BUDLONG AV,34.0328,-118.2959,3604,1280,286552,8522,47
2117587,190516898,2019-10-15,2019-10-14 18:00:00,5,Harbor,526,1,330,BURGLARY FROM VEHICLE,0321 0377 1300 0344,...,-1,-1,800 LAGOON AV,33.7797,-118.2675,9861,1377,20564,45552,47


## Crime dimension
* Next up the dimension of the crime commited will be made
* Particularly we take the columns `Crm Cd` and `Crm Cd Desc` `Part 1-2` and clear the duplicate values
* After checking that each crime code corresponds to exactly one crime description and either part 1 or 2 crime type, there is no extra work to do.
* The results are saved in a new dataframe called `crime_dimension`

In [76]:
la_crimes[['Crm Cd','Crm Cd Desc','Part 1-2']].drop_duplicates()['Crm Cd Desc'].value_counts() #check

Crm Cd Desc
VIOLATION OF COURT ORDER                        1
CHILD PORNOGRAPHY                               1
RECKLESS DRIVING                                1
ABORTION/ILLEGAL                                1
CONTRIBUTING                                    1
                                               ..
SHOPLIFTING - PETTY THEFT ($950 & UNDER)        1
VANDALISM - MISDEAMEANOR ($399 OR UNDER)        1
BIKE - STOLEN                                   1
THEFT, COIN MACHINE - GRAND ($950.01 & OVER)    1
BLOCKING DOOR INDUCTION CENTER                  1
Name: count, Length: 142, dtype: int64

In [77]:
la_crimes[['Crm Cd','Crm Cd Desc','Part 1-2']].drop_duplicates()['Crm Cd'].value_counts()#check

Crm Cd
900    1
814    1
438    1
952    1
954    1
      ..
442    1
745    1
480    1
473    1
432    1
Name: count, Length: 142, dtype: int64

In [78]:
crime_dimension = la_crimes[['Crm Cd','Crm Cd Desc','Part 1-2']].drop_duplicates()

* Moreover, we add the crime code `Crm Cd` $-1$ as a code for no crime . This will be used for the the columns `Crm Cd 2`, `Crm Cd 3` and `Crm Cd 4` (instead of them being null) because some crime incidents only include one primary crime

In [79]:
crime_dimension = crime_dimension.append({'Crm Cd':-1, 'Crm Cd Desc': 'NO CRIME',  'Part 1-2': np.nan}, ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'append'

* Now we have to make sure that even the `Crm Cd 2` `Crm Cd 3` and `Crm Cd 4` codes are included in the crime dimension.
* For that we create a new dataframe that that includes all  the unique `Crm Cd 2` `Crm Cd 3` `Crm Cd 4` codes

In [ ]:
crm2 = la_crimes['Crm Cd 2'].drop_duplicates()
crm3 = la_crimes['Crm Cd 3'].drop_duplicates()
crm4 = la_crimes['Crm Cd 4'].drop_duplicates()
crm2 = crm2.append(crm3, ignore_index=True).drop_duplicates()
crm2 = crm2.append(crm4, ignore_index=True).drop_duplicates()
crm2

0       -1
1      998
2      930
3      900
4      940
      ... 
142    438
143    652
144     99
145    978
146    884
Length: 147, dtype: int64

In [ ]:
crm_sec_codes= pd.DataFrame()
crm_sec_codes['Crm Cd'] = crm2.copy()
crm_sec_codes

,Crm Cd
0,-1
1,998
2,930
3,900
4,940
...,...
142,438
143,652
144,99
145,978


* Now we can merge the crimecode dimension with the additional codes
* For the codes that we do not have any crime description, we change their `Crm Cd Desc` to `'No information'`

In [ ]:
crime_dimension = pd.merge(crime_dimension, crm_sec_codes, on='Crm Cd', how='outer')
crime_dimension['Crm Cd Desc'] = crime_dimension['Crm Cd Desc'].fillna('No information')
crime_dimension

,Crm Cd,Crm Cd Desc,Part 1-2
0,900,VIOLATION OF COURT ORDER,2.0
1,901,VIOLATION OF RESTRAINING ORDER,2.0
2,928,THREATENING PHONE CALLS/LETTERS,2.0
3,930,CRIMINAL THREATS - NO WEAPON DISPLAYED,2.0
4,956,"LETTERS, LEWD - TELEPHONE CALLS, LEWD",2.0
...,...,...,...
161,972,No information,NaN
162,953,No information,NaN
163,945,No information,NaN
164,99,No information,NaN


## Premises dimension
* Now the dimension of the premises of the crime will be made. In detail, those are the type of structure, vehicle, or location where the crime took place.
* Particularly we take the columns `Premis Cd` and `Premis Desc` and clear the duplicate values

In [ ]:
la_crimes[['Premis Cd','Premis Desc']].drop_duplicates()['Premis Desc'].value_counts()

RETIRED (DUPLICATE) DO NOT USE THIS CODE    3
STAPLES CENTER *                            1
DRIVE THRU*                                 1
FURNITURE STORE                             1
MARKET                                      1
                                           ..
MTA - GREEN LINE - AVALON                   1
WEBSITE                                     1
MTA - GOLD LINE - HIGHLAND PARK             1
FOSTER HOME BOYS OR GIRLS*                  1
METROLINK TRAIN                             1
Name: Premis Desc, Length: 320, dtype: int64

* We see that different premises codes correspond to the premis desc `RETIRED (DUPLICATE) DO NOT USE THIS CODE`
* Because the premise description says not to be used, we will changed all those premises codes and descriptions to 0 and 'unknown' respectively

In [ ]:
indexes_premis_retired = la_crimes.loc[la_crimes['Premis Desc']== 'RETIRED (DUPLICATE) DO NOT USE THIS CODE'].index
la_crimes.loc[indexes_premis_retired,'Premis Desc'] = 'unknown'
la_crimes.loc[indexes_premis_retired,'Premis Cd'] = 0

* Each premises code corresponds to exactly one premises description
* The results are saved in a new dataframe called `premises_dimension`

In [ ]:
premises_dimension = la_crimes[['Premis Cd','Premis Desc']].drop_duplicates()

## Weapon dimension
Next up the dimension of the weapon used will be created
* Particularly we take the columns `Weapon Used Cd` and `Weapon Desc` and clear the duplicate values
* The results are saved in a new dataframe called `weapon_dimension`

In [ ]:
la_crimes[['Weapon Used Cd','Weapon Desc']].drop_duplicates()['Weapon Desc'].value_counts()

unknown              2
AXE                  1
HAMMER               1
HAND GUN             1
PIPE/METAL PIPE      1
                    ..
FIXED OBJECT         1
ROPE/LIGATURE        1
PHYSICAL PRESENCE    1
KITCHEN KNIFE        1
BOARD                1
Name: Weapon Desc, Length: 80, dtype: int64

* We see that different weapon codes correspond to the weapon desc `unknown`
* Because we have used the weapon code $0$ for null values to describe incidents where we do not know the weapon, we will change the other weapon code ($222$) to 0 as well.

In [ ]:
la_crimes.loc[la_crimes['Weapon Desc']== 'unknown']['Weapon Used Cd'].value_counts()

0      1404460
222          1
Name: Weapon Used Cd, dtype: int64

In [ ]:
index_weaponcd222 = la_crimes.loc[la_crimes['Weapon Used Cd']== 222].index
la_crimes.loc[index_weaponcd222,'Weapon Desc'] = 0

* Each weapon code corresponds to exactly one weapon description
* The results are saved in a new dataframe called `weapon_dimension`

In [ ]:
weapon_dimension = la_crimes[['Weapon Used Cd','Weapon Desc']].drop_duplicates()

## Status dimension
* Next up the dimension of the status of the crime is made
* Particularly we take the columns `Status` and `Status Desc` and clear the duplicate values
* After checking that each status code corresponds to exactly one status description, there is no extra work to do.
* The results are saved in a new dataframe called `status_dimension`

In [ ]:
la_crimes[['Status','Status Desc']].drop_duplicates()

,Status,Status Desc
0,AA,Adult Arrest
1,AO,Adult Other
4,IC,Invest Cont
172,JO,Juv Other
557,JA,Juv Arrest
158469,CC,UNK


In [ ]:
status_dimension = la_crimes[['Status','Status Desc']].drop_duplicates()

## Coordinate dimension
* Next up is the dimension of the geographic coordinates of the incident
* Particularly we take the columns `LON` and `LAT` and clear the duplicate values
* The results are saved in a new dataframe called `coordinates_dimension`
* We save the index of the new dataframe as a new column `COORD_CD`

In [ ]:
coordinates_dimension = la_crimes[['LON','LAT']].drop_duplicates()
coordinates_dimension = coordinates_dimension.reset_index(drop=True)
coordinates_dimension['COORD_CD'] = coordinates_dimension.index
coordinates_dimension

,LON,LAT,COORD_CD
0,-118.2695,33.9825,0
1,-118.2696,34.0007,1
2,-118.2916,34.0509,2
3,-118.2891,34.0508,3
4,-118.4400,34.1794,4
...,...,...,...
64482,-118.6071,34.2636,64482
64483,-118.6138,34.1848,64483
64484,-118.2894,33.9088,64484
64485,-118.4209,34.2301,64485


* We will use the index column of the new dataframe `coordinates_dimension` as a foreign key and thus we need to add a new column to `la_crimes` containing for each incident the value of this foreign key
* After we do that, the columns `LAT` and `LON` are no longer needed in the dataframe `la_crimes` as we have the mapping from the column `COORD_CD` to the dimension table `coordinates_dimension`

In [ ]:
la_crimes = pd.merge(la_crimes, 
               coordinates_dimension, 
               on=['LON','LAT'],
               how='inner')
la_crimes

,DR_NO,Date Rptd,DATE OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Crm Cd 4,LOCATION,LAT,LON,LOCATION_CD,DATE_RPTD_CD,DATE_OCC_CD,MOCODE_CD,VICTIM_CD,COORD_CD
0,1307355,2010-02-20,2010-02-20 13:50:00,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,...,-1,300 E GAGE AV,33.9825,-118.2695,0,0,0,0,0,0
1,121321665,2012-10-27,2012-10-27 16:15:00,13,Newton,1385,1,440,THEFT PLAIN - PETTY ($950 & UNDER),0344,...,-1,300 E GAGE AV,33.9825,-118.2695,0,15,4221,4,0,0
2,171318398,2017-08-14,2017-08-11 06:00:00,13,Newton,1395,1,331,THEFT FROM MOTOR VEHICLE - GRAND ($400 AND OVER),0344,...,-1,300 E GAGE AV,33.9825,-118.2695,68762,426,11219,4,0,0
3,151312312,2015-05-30,2015-05-30 17:00:00,13,Newton,1395,1,440,THEFT PLAIN - PETTY ($950 & UNDER),0344,...,-1,6300 S SAN PEDRO ST,33.9825,-118.2695,33025,963,212429,4,0,0
4,121314885,2012-07-02,2012-07-02 13:40:00,13,Newton,1383,2,624,BATTERY - SIMPLE ASSAULT,0416,...,-1,SAN PEDRO,33.9825,-118.2695,33504,3271,42901,66,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114694,131704213,2013-01-05,2013-01-05 18:30:00,17,Devonshire,1721,1,310,BURGLARY,1609 0344,...,-1,22000 LINFIELD AV,34.2636,-118.6071,93185,748,185385,5461,28,64482
2114695,172109827,2017-04-25,2017-04-25 10:00:00,21,Topanga,2155,2,662,"BUNCO, GRAND THEFT",0800 1202 1822,...,-1,SYLVAN ST,34.1848,-118.6138,113827,1170,120122,46817,28,64483
2114696,161807341,2016-03-08,2016-03-06 18:00:00,18,Southeast,1871,2,940,EXTORTION,0443 0432 1906 1823 0513 0514 0520 0528,...,-1,3700 S MENLO,33.9088,-118.2894,105196,1060,118406,169640,28,64484
2114697,191917901,2019-10-14,2019-10-14 14:30:00,19,Mission,1988,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",0416 0913 1218 0447 0340 2004 0345 0334 0344 1223,...,-1,13100 WENTWORTH ST,34.2301,-118.4209,51106,1390,20547,45099,31,64485


## Removing dimensions from the fact table-dataframe
* Now we can drop the extra columns that there are on our fact table 
* Particularly, many columns are dropped because we already made the dimensions dataframes which contain the codes and the description for each dimension
* We are going to keep the codes of the dimensions and whenever we need some extra information, we can look up the dimension tables

In [ ]:
la_crimes = la_crimes.drop(['AREA','AREA NAME','LOCATION','Rpt Dist No','Date Rptd', 'Crm Cd Desc', 'Part 1-2',
                            'Premis Desc', 'DATE OCC', 'Mocodes','Vict Sex', 'Vict Descent',
                            'Weapon Desc', 'Status Desc','LON','LAT'], axis=1)
la_crimes

,DR_NO,Crm Cd,Vict Age,Premis Cd,Weapon Used Cd,Status,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION_CD,DATE_RPTD_CD,DATE_OCC_CD,MOCODE_CD,VICTIM_CD,COORD_CD
0,1307355,900,48,501,0,AA,-1,-1,-1,0,0,0,0,0,0
1,121321665,440,43,119,0,IC,-1,-1,-1,0,15,4221,4,0,0
2,171318398,331,34,104,0,IC,-1,-1,-1,68762,426,11219,4,0,0
3,151312312,440,33,101,0,IC,-1,-1,-1,33025,963,212429,4,0,0
4,121314885,624,16,102,400,IC,-1,-1,-1,33504,3271,42901,66,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114694,131704213,310,43,501,0,IC,-1,-1,-1,93185,748,185385,5461,28,64482
2114695,172109827,662,60,101,0,IC,-1,-1,-1,113827,1170,120122,46817,28,64483
2114696,161807341,940,22,501,0,IC,-1,-1,-1,105196,1060,118406,169640,28,64484
2114697,191917901,230,37,102,311,IC,-1,-1,-1,51106,1390,20547,45099,31,64485


Renaming the coumns name for easier use

In [ ]:
la_crimes.rename(columns={'Crm Cd':'CRM_CD',
                         'Vict Age':'VICT_AGE',
                         'Premis Cd':'PREMIS_CD','Weapon Used Cd':'WEAPON_CD',
                         'Status':'STATUS','Crm Cd 2':'CRM_CD2',
                         'Crm Cd 3':'CRM_CD3', 'Crm Cd 4':'CRM_CD4'},
                         inplace = True)

In [ ]:
location_dimension.rename(columns={'Rpt Dist No': 'RPT_DIST_NO','AREA NAME':'AREA_NAME','LOCATION':'LOCATION_NAME'}
                          ,inplace = True)
crime_dimension.rename(columns={'Crm Cd': 'CRM_CD','Crm Cd Desc':'CRM_DESCR','Part 1-2':'PART_1_2'},inplace = True)
premises_dimension.rename(columns={'Premis Cd': 'PREMIS_CD','Premis Desc':'PREMIS_DESCR'},inplace = True)
weapon_dimension.rename(columns={'Weapon Used Cd':'WEAPON_CD', 'Weapon Desc':'WEAPON_DESCR'},inplace = True)
status_dimension.rename(columns={'Status': 'STATUS','Status Desc':'STATUS_DESCR'},inplace = True)
date_occ_dimension.rename(columns={'DATE OCC': 'DATE_OCC'},inplace = True)
date_rptd_dimension.rename(columns={'Date Rptd': 'DATE_RPTD'},inplace = True)
victim_dimension.rename(columns={'Vict Sex': 'VICT_SEX','Vict Descent':'VICT_DESCENT'},inplace = True)
suspect_dimension.rename(columns={'Mocodes': 'MOCODES'},inplace = True)

Saving the fact table(dataframe) and dimension tables(dataframes) as  csv documents in order to insert them into the database (not necessary for data analysis)

la_crimes.to_csv('la_crimes_cleared.csv',index=False)
location_dimension.to_csv('location_dimension.csv',index=False)
crime_dimension.to_csv('crime_dimension.csv',index=False)
premises_dimension.to_csv('premises_dimension.csv',index=False)
weapon_dimension.to_csv('weapon_dimension.csv',index=False)
status_dimension.to_csv('status_dimension.csv',index=False)
date_occ_dimension.to_csv('date_occ_dimension.csv',index=False)
date_rptd_dimension.to_csv('date_rptd_dimension.csv',index=False)
victim_dimension.to_csv('victim_dimension.csv',index=False)
suspect_dimension.to_csv('suspect_dimension.csv',index=False)
coordinates_dimension.to_csv('coordinates_dimension.csv',index=False)